## Network Science Project



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import requests 
import pandas as pd 
import time

In [ ]:
os.environ['TOKEN'] = "AAAAAAAAAAAAAAAAAAAAANxrUwEAAAAAFs4Es89FbZJFOZQGf7ABUq9SxzU%3DFZ2fsqYCdnLsaF1wgRiGYBCC3EgNOnKHe1caVHr8EdFh1WPZgw"

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

headers = create_headers(os.environ['TOKEN'])

In [ ]:
def create_url(keyword, start_date, end_date, endpoint):
    
    search_url = endpoint #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    #also can request different fields
    query_params = {'query': keyword,
                    'max_results': 100,
                    'start_time': start_date,
                    'end_time': end_date,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source,context_annotations,entities',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'pagination_token': {}}
    return (search_url, query_params)

In [ ]:
def connect_to_endpoint(url, headers, params, next_token = None):
    print(next_token)
    if next_token is not None and next_token != '':
      params['next_token'] = next_token
    response = requests.request("GET", url, headers = headers, params = params)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [ ]:
path = "/content"

In [ ]:
def get_data(keyword, start_time, end_time, next_token, endpoint):
  results = []
  

  while next_token is not None:
    try:     
      url = create_url(keyword, start_time, end_time, endpoint)
      json_response = connect_to_endpoint(url[0], headers, url[1], next_token)

      if "data" in json_response:
        results.extend(json_response["data"])
        
      if "meta" in json_response:
        if "next_token" in json_response["meta"].keys():
          next_token = json_response["meta"]["next_token"]
          
        else:
          next_token = None
      else:
        next_token = None

      #uncomment to save by batches - useful if error occurs
      print(str(len(results)) + "Tweets for keyword " + keyword + " downloaded in this batch.")
      batch = pd.DataFrame(results)
      #batch.to_csv(path + keyword + "_" + start_time + end_time + ".csv", index=False)
      batch.to_pickle(path + keyword + "_" + start_time + "_" + end_time + ".pkl")
      
      time.sleep(3)
    except Exception as e:
      print("Error occured", e)
      print(e)
    next_token=None
  print("Done")
  # results.to_csv(path + keyword+ "_" + start_time + end_time + "_full.csv", index=False)
  # results.to_pickle(path + start_time + end_time + "_full.pkl")
  return (next_token)

In [ ]:
query = "#ukrainewar"
#FOR START AND END TIME THIS IS THE DATE FORMAT 2022-02-01T00:00:00Z (YYYY-MM-DDTHH:mm:ssZ)
start_time = "2022-03-01T00:00:00.000Z"
end_time = "2022-04-02T00:00:00.000Z"

In [ ]:
next_token = get_data(query, start_time=start_time, end_time=end_time, endpoint="https://api.twitter.com/2/tweets/search/all", next_token="")